In [ ]:
!pip install -i https://test.pypi.org/simple/ Vampyr-MTL-Max-JJ==0.0.5

Looking in indexes: https://test.pypi.org/simple/


In [ ]:
from Vampyr_MTL import functions
import numpy as np
import math
from scipy import linalg
import plotly.express as px

In [ ]:
clus_var = 900
task_var = 16
nois_var = 150

clus_num = 2
clus_task_num = 10
task_num = clus_num * clus_task_num
sample_size = 100
dimension = 20
comm_dim = 2
clus_dim = math.floor((dimension - comm_dim)/2)

# generate cluster model
cluster_weight = np.random.randn(dimension, clus_num)* clus_var
for i in range(clus_num):
    bll = np.random.permutation(range(dimension-clus_num))<=clus_dim
    blc = np.array([False]*clus_num)
    bll = np.hstack((bll, blc))
    cluster_weight[:,i][bll]=0
cluster_weight[-1-comm_dim:, :]=0
W = np.tile(cluster_weight, (1, clus_task_num))
cluster_index = np.tile(range(clus_num), (1, clus_task_num)).T

# generate task and intra-cluster variance
W_it = np.random.randn(dimension, task_num) * task_var
for i in range(task_num):
    bll = np.hstack(((W[:-1-comm_dim+1,i]==0).reshape(1,-1), np.zeros((1,comm_dim))==1))
    W_it[:,i][bll.flatten()]=0
W = W+W_it

W = W + np.random.randn(dimension, task_num)*nois_var

X = [0]*task_num
Y = [0]*task_num
for i in range(task_num):
    X[i] = np.random.randn(sample_size, dimension)
    xw = X[i] @ W[:,i]
    s= xw.shape
    xw = xw + np.random.randn(s[0]) * nois_var
    Y[i] = np.sign(xw)

In [ ]:
from Vampyr_MTL.functions.MTL_Cluster_Least_L21 import MTL_Cluster_Least_L21
from Vampyr_MTL.evaluations.utils import opts as op

In [ ]:
opts = op(1500,2)
clf = MTL_Cluster_Least_L21(opts,2)
clf.fit(X, Y)
corr = clf.analyse()

outer loop:   0%|          | 0/1500 [00:00<?, ?it/s]
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
outer loop:   0%|          | 3/1500 [00:00<00:51, 29.12it/s]
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.00/1.00k [00:00<?, ?it/s]
                                                        
inner loop:   0%|          | 0.

In [ ]:
fig = px.imshow(corr, color_continuous_scale='Bluered_r')
fig.update_layout(
title={
    'text': "predict",
    })
fig.show()
OrderedTrueModel = np.zeros(W.shape)
clus_task_num = task_num//clus_num
for i in range(clus_num):
    clusModel = W[:, i:task_num:clus_num]
    OrderedTrueModel[:, (i)*clus_task_num: (i+1)* clus_task_num] = clusModel
corr2 = 1-np.corrcoef(OrderedTrueModel)
fig2 = px.imshow(corr2, color_continuous_scale='Bluered_r')
fig2.update_layout(
title={
    'text': "real",
    })
fig2.show()

In [ ]:
fig = px.imshow(clf.get_weights(), color_continuous_scale='Bluered_r')
fig.update_layout(
title={
    'text': "predict",
    })
fig.show()
fig2 = px.imshow(W, color_continuous_scale='Bluered_r')
fig2.update_layout(
title={
    'text': "real",
    })
fig2.show()